In [2]:
import json
import pandas as pd
import re
from textblob import TextBlob
from ast import literal_eval
from datetime import datetime

In [3]:
# funcion para corregir y cargar cada linea del archivo 'australian_user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# lista para almacenar los datos desanidados
data_list = []

# exprecion regular para extraer números de la columna 'funny'
numero_regex = re.compile(r'\d+')

# lee el archivo 'australian_user_reviews.json' original, corrige y procesa cada línea
with open('australian_user_reviews.json', 'r', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            entrada = cargar_y_corregir_linea(linea)
            user_id = entrada['user_id']
            user_url = entrada['user_url']
            # iteracion sobre cada reseña
            for reseña in entrada['reviews']:
                # estrae el numero de la columna 'funny'
                funny_valor = re.search(numero_regex, reseña.get('funny', ''))
                if funny_valor:
                    funny = int(funny_valor.group())
                else:
                    funny = None
                
                # elimina 'Posted' de la columna 'posted'
                posted = reseña.get('posted', '').replace('Posted ', '', 1)

                reseña_dict = {
                    'user_id': user_id,
                    'user_url': user_url,
                    'funny': funny,
                    'posted': posted,
                    'item_id': int(reseña.get('item_id', '')),  # convierte a entero
                    'helpful': reseña.get('helpful', ''),
                    'recommend': reseña.get('recommend', ''),
                    'review': reseña.get('review', '')  # mantiene el texto original de la review
                }
                data_list.append(reseña_dict)
        except json.JSONDecodeError as e:
            None

# crea el DataFrame
df = pd.DataFrame(data_list)

In [2]:
# guarda el DataFrame modificado en un nuevo archivo CSV
df.to_csv('user_reviews.csv', index=False)

# guarda el DataFrame modificado en un nuevo archivo Parquet
df.to_parquet('user_reviews.parquet', index=False)

In [3]:
df.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'item_id', 'helpful',
       'recommend', 'review'],
      dtype='object')

In [6]:
def get_sentiment_with_recommend(recommend, review):
    if isinstance(review, str):  # Verifica si el tipo de review es string
        if recommend:  # Verifica si la recomendación es verdadera
            analysis = TextBlob(str(review))  # Convierte a cadena y analiza el texto de la reseña
            polarity = analysis.sentiment.polarity
            if polarity > 0:
                return 2  # Positivo si polaridad > 0
            else:
                return 1  # Neutral si polaridad <= 0 y recommend true
        else:
            return 0  # No recomendado, se asume neutral o negativo
    else:
        return 1  # No hay reseña, se asume neutral
# Convertir la columna 'review' a cadena (str)
df['review'] = df['review'].astype(str)

# Aplica la función a cada review
df['sentiment_analysis'] = df.apply(lambda x: get_sentiment_with_recommend(x['recommend'], x['review']), axis=1)

# Reemplaza la columna 'review' con la nueva columna 'sentiment_analysis'
df.drop('review', axis=1, inplace=True)
df.rename(columns={'sentiment_analysis': 'review'}, inplace=True)

print(df)

                 user_id                                           user_url  \
0      76561198079601835  http://steamcommunity.com/profiles/76561198079...   
1            MeaTCompany           http://steamcommunity.com/id/MeaTCompany   
2      76561198089393905  http://steamcommunity.com/profiles/76561198089...   
3      76561198089393905  http://steamcommunity.com/profiles/76561198089...   
4      76561198156664158  http://steamcommunity.com/profiles/76561198156...   
...                  ...                                                ...   
29633  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
29634  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
29635        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
29636        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
29637        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

       funny             posted  item_id  \
0      

In [4]:
# Reemplaza los valores NaN por None en el DataFrame
df = df.where(df.notna(), None)

In [17]:
from datetime import datetime

# Definimos una función para extraer y convertir la fecha
def extract_date(posted_str):
    # Elimina la palabra 'Posted' y espacios en blanco al principio y al final
    date_str = posted_str.replace('Posted', '').replace('.', '').strip()
    try:
        # Intenta convertir la cadena de fecha en un objeto de fecha utilizando datetime.strptime
        date_obj = datetime.strptime(date_str, '%B %d, %Y').date()
    except ValueError:
        try:
            # Intenta nuevamente con el formato '%B %d' si la conversión falla
            date_obj = datetime.strptime(date_str, '%B %d').date()
        except ValueError:
            # Si no se puede convertir a ninguna de las dos formas, devuelve None
            date_obj = None
    except TypeError:
        # Si ocurre un error de tipo al intentar convertir la fecha, devuelve None
        date_obj = None

    return date_obj

# Ejemplo de uso
date_str = 'Posted November 5, 2011.'
date_obj = extract_date(date_str)
print(date_obj)



2011-11-05


In [18]:
df.to_csv('reviews_sentiment_analysis.csv', index=False)

In [19]:
df.columns

Index(['user_id', 'user_url', 'funny', 'item_id', 'helpful', 'recommend',
       'review', 'posted'],
      dtype='object')

In [5]:
df_user_recomendacion = df[['user_id', 'recommend']]

# Eliminar filas con muchos datos faltantes
df_user_recomendacion = df_user_recomendacion.dropna(thresh=1)

df_user_recomendacion

,user_id,recommend
0,76561198079601835,True
1,MeaTCompany,True
2,76561198089393905,True
3,76561198089393905,True
4,76561198156664158,True
...,...,...
29633,76561198312638244,True
29634,76561198312638244,True
29635,LydiaMorley,True
29636,LydiaMorley,True


In [6]:
df_user_recomendacion.to_csv('user-recomendacion.csv', index=False)

In [30]:
# Cargar las tablas pequeñas
df_id_gamer_price = pd.read_csv('id-gamer-price.csv')
#df_user_recomendaciones = pd.read_csv('user-recomendaciones.csv')
df_user_items = pd.read_csv('user-items.csv')

# Combinar las tablas df_user_items y df_id_gamer_price en base a 'item_id'
df_combined = pd.merge(df_user_items, df_id_gamer_price, on='item_id')

# Combinar la tabla combinada anterior con df_user_recomendaciones en base a 'user_id'
df_combined = pd.merge(df_combined, df_user_recomendacion, on='user_id')

# Verificar la tabla combinada resultante
print(df_combined.head())


             user_id  item_id         price  recommend
0  76561198077992470   105600          9.99       True
1  76561198077992470   230410  Free to Play       True
2  76561198077992470   301520  Free to Play       True
3  76561198077992470   304930  Free to Play       True
4  76561198077992470      730         14.99       True


In [31]:
import pandas as pd

# Reemplazar 'free' por 0 en la columna 'price'
df_combined['price'] = df_combined['price'].replace(to_replace='.*free.*', value='0', regex=True)

# Convertir la columna 'price' a tipo float, manejar excepciones para valores no numéricos
def try_convert_to_float(value):
    try:
        return float(value)
    except ValueError:
        return 0.0  # Asignar 0 para valores no numéricos

df_combined['price'] = df_combined['price'].apply(try_convert_to_float)

# Verificar los cambios
print(df_combined.head())




             user_id  item_id  price  recommend
0  76561198077992470   105600   9.99       True
1  76561198077992470   230410   0.00       True
2  76561198077992470   301520   0.00       True
3  76561198077992470   304930   0.00       True
4  76561198077992470      730  14.99       True


In [33]:
df_combined['recommend'] = df_combined['recommend'].replace({'True': True, 'False': False})

In [34]:
df_combined.to_csv('user-data-price.csv', index=False)